# Initialization of the notebook

In [ ]:
# this should be the set to false for production use
# and set to true for testing (default) but can be already set to false from outside (e.g., by the test script)

if "doRunTests" not in globals():
    doRunTests = True

# Utility functions

## Split a file name into its components

In [ ]:
def file_parts(fullFname):
    """Split a full filename into path, filename and extension"""
    fpath = os.path.dirname(fullFname)
    fname = os.path.basename(fullFname)
    fname, extension = os.path.splitext(fname)
    return fpath, fname, extension

## Read a marker csv file

We have to read a marker csv file and return a list of [timestamp, marker] pairs.   
We want to read both kinect and mouse marker csv files, but the kinect markers lack the timestamp column.   


In [ ]:
import numpy as np
from datetime import datetime
import tempfile


def readMarkerCsv(fnameMarkerCsv):
    """Read a marker csv file and return a list of [timestamp, marker] pairs

    Parameters
    ----------
    fnameMarkerCsv : str
        Full filename of the marker csv file

    Returns
    -------

    out : list
        List of [timestamp, marker] pairs
            timestamp : float (in seconds)
            marker : str
    """

    with open(fnameMarkerCsv, "r") as fname:
        txt = fname.readlines()

    lines = [line.split(",") for line in txt]

    # PROBLEM: kinect markers lack the timestamp column
    # a marker is a line with 3 tokens : datetime,timestamp,marker \n
    # ACTION : add the timestamp column if we have only 2 columns
    for i in range(len(lines)):
        if len(lines[i]) == 2 and lines[i][0][0].isdigit():
            # create a timestamp that is in milliseconds (as in mouse markers)
            timestamp = (
                datetime.strptime(lines[i][0], "%Y-%m-%d %H:%M:%S.%f").timestamp()
                * 1000
            )
            lines[i].insert(1, str(timestamp))
            txt[i] = ",".join(lines[i])

    # PROBLEM: mouse markers lack the quotes around the multiline markers
    # find the lines where token 3 is "\n" = start of a multiline marker
    for i in range(len(lines)):
        line = lines[i]
        # find the start of a multiline marker
        if len(line) == 3 and line[2] == "\n":
            # add a " before the end of the line
            txt[i] = txt[i][:-1] + '"\n'
            # find the end of the multiline marker
            for j in range(i + 1, len(lines)):
                # if we have a normal one-line-marker
                if len(lines[j]) == 3:
                    # add a " before the end of the line (of the previous line)
                    txt[j - 1] = txt[j - 1][:-1] + '"\n'
                    break
                # if are at the end of the file
                if j == len(lines) - 1 and len(lines[j]) != 3:
                    # add a " before the end of the line
                    txt[j] = txt[j][:-1] + '"\n'
                    break

    # write the modified file to a temporary file and load it with np.loadtxt
    with tempfile.NamedTemporaryFile(mode="w", delete=False) as fname:
        fname.writelines(txt)
        tempFileName = fname.name
    lines = np.loadtxt(
        tempFileName, skiprows=3, delimiter=",", quotechar='"', dtype=str
    )

    # remove the first column (we shall need only the timestamp and the marker to compare with the xdf file)
    lines = np.delete(lines, 0, 1)

    # return a list of [timestamp, marker] pairs
    out = []
    for line in lines:
        # CAUTION : the timestamp must be in seconds (as in xdf files)
        timestamp = float(line[0]) / 1000
        marker = line[1]
        out.append([timestamp, marker])

    return out


if doRunTests:
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_l_m_np.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Reaching/ReArm_C1P02_20210322_1_r_k_m.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_k_m.csv"

    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Circle/ReArm_C1P02_20210322_V1_c_k_m.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Circle/ReArm_C1P02_20210322_V1_c_l_m_np.csv"

    lines = readMarkerCsv(fnameMarkerCSV)

    # print the lines
    for line in lines:
        print(line)

# Compute the time delays

## Compute the xdf-to-csv time delay for the Kinect data


In [ ]:
import os
import pyxdf
import matplotlib.pyplot as plt


def getKinectDelayInfos(xdf_fullFname):
    """
    Get the delay between the XDF data and the corresponding CSV file for the kinect data

    """
    fpath, fname, extension = file_parts(xdf_fullFname)
    if extension != ".xdf":
        return None

    # NOTE: There is a bug in the kinect markers stream: the csv is not exactly the same...
    #       So we shall use the MoCap stream to get the delay (
    #       hopefully it is the same, and there is a kinect MoCap csv file for each xdf file
    # load the EuroMov-Mocap-Kinect stream
    data, header = pyxdf.load_xdf(
        filename=xdf_fullFname,
        select_streams=[{"type": "MoCap", "name": "EuroMov-Mocap-Kinect"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    kinectStream = data[0]

    # get the corresponding kinect file name
    tokens = fname.split("_")
    # keep the common part of the filename
    kinectFname = "_".join(tokens[:5])
    # add the end of the filename
    kinectFname = kinectFname + "_k.csv"

    fullFname_csv = os.path.join(fpath, kinectFname)

    # get the corresponding data from the kinect CSV file
    data_csv = np.loadtxt(fullFname_csv, skiprows=3, delimiter=",", dtype=float)

    # get the timestamps from the two files
    timestampCSV_sec = data_csv[:, 0] / 1000
    timestampXDF_sec = kinectStream["time_stamps"]

    # get the (median) delay between the two files
    median_delay = np.median(timestampXDF_sec - timestampCSV_sec)

    # get the distribution of the delays
    delays_distribution = timestampXDF_sec - timestampCSV_sec - median_delay

    return {
        "median_delay": median_delay,
        "delays_distribution": delays_distribution,
        "xdf_fullFname": xdf_fullFname,
        "csv_fullFname": fullFname_csv,
    }


def test_delays(delays):
    # print the delays
    print("Delays:")
    for delay in delays:
        print(delay[0]["median_delay"], delay[0]["xdf_fullFname"])

    # make a boxplot of the distribution of the delays
    fig = plt.figure()
    toBoxplot = []
    labelBoxplot = []
    for delay in delays:
        xdf_fname = os.path.basename(delay[0]["xdf_fullFname"])
        csv_fname = os.path.basename(delay[0]["csv_fullFname"])
        val = delay[0]["delays_distribution"] * 1000  # in milliseconds
        nVal = len(val)
        toBoxplot.append(val)
        labelBoxplot.append("{}\n{}\n{} values".format(xdf_fname, csv_fname, nVal))
    plt.boxplot(toBoxplot, vert=False, labels=labelBoxplot)
    plt.gca().yaxis.set_ticks_position("right")
    plt.xlabel("Delay (milliseconds)")
    plt.title("Delays relative to the median delay")
    plt.show()

    # make a plot of the distribution of the delays
    fig = plt.figure()
    for delay in delays:
        xdf_fname = os.path.basename(delay[0]["xdf_fullFname"])
        csv_fname = os.path.basename(delay[0]["csv_fullFname"])
        plt.plot(
            delay[0]["delays_distribution"] * 1000,
            "o",
            label=csv_fname,
        )
    plt.ylabel("Delay (milliseconds)")
    plt.xlabel("Line number")
    plt.title("Delays relative to the median delay")
    # put the legend on the top out of the plot
    plt.legend(bbox_to_anchor=(0.5, 1.1), loc="lower center", ncol=3)
    plt.show()


if doRunTests:
    visit_path = "../dat/ReArm.lnk/ReArm_C1P02"
    visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210820_V2"

    fullFname_goodFiles = os.path.join(visit_path, "goodFiles.log")

    # load goodFiles.txt into a list
    goodFiles = []
    with open(fullFname_goodFiles, "r") as f:
        for line in f:
            goodFiles.append(line.strip())
    # get the delay for all the files
    delays = []

    for file_path in goodFiles:
        fullFname = file_path
        if not os.path.isabs(file_path):
            fullFname = os.path.join(visit_path, file_path)
        delay_infos = getKinectDelayInfos(fullFname)
        if delay_infos is not None:
            delays.append([delay_infos, file_path])

    test_delays(delays)

## Compute the xdf-to-csv time delay for the Mouse data

In [ ]:
def getMouseDelayInfos(xdf_fullFname):
    """
    Get the delay between the XDF file and the corresponding CSV file for the mouse data
    """

    fpath, fname, extension = file_parts(xdf_fullFname)

    tk = fname.split("_")

    if extension != ".xdf":
        # print("Error: the file extension is not .xdf")
        return None
    # load the EuroMov-Mocap-Kinect stream
    data, header = pyxdf.load_xdf(
        filename=xdf_fullFname,
        select_streams=[{"type": "Markers", "name": "Mouse"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    data_XDF = data[0]

    # get the corresponding mouse file name
    if tk[4] == "r":
        mouseFnameEnd = "_l_m_sau_p.csv"
    elif tk[4] == "c":
        mouseFnameEnd = "_l_m_p.csv"

    mouseFname = (
        tk[0] + "_" + tk[1] + "_" + tk[2] + "_" + tk[3] + "_" + tk[4] + mouseFnameEnd
    )

    fullFname_csv = os.path.join(fpath, mouseFname)
    mouse_csv = readMarkerCsv(fullFname_csv)
    csv_timestamps = np.array([float(line[0]) for line in mouse_csv])

    # restrict the xdf data to the nb of csv data
    # NOTE: The xdf data contains extra markers from the kinect
    # This is a bug, as xdf and csv should contain the same markers...
    nCsv = len(csv_timestamps)
    xdf_timestamps = data_XDF["time_stamps"][:nCsv]

    # get the timestamps in seconds
    # CAUTION: do not forget to check this if you change the code before this line
    timestampCSV_sec = csv_timestamps
    timestampXDF_sec = xdf_timestamps

    # get the (median) delay between the two files
    median_delay = np.median(timestampXDF_sec - timestampCSV_sec)

    # get the distribution of the delays
    delays_distribution = timestampXDF_sec - timestampCSV_sec - median_delay

    return {
        "median_delay": median_delay,
        "delays_distribution": delays_distribution,
        "xdf_fullFname": xdf_fullFname,
        "csv_fullFname": fullFname_csv,
    }


if doRunTests:
    # get the delay for all the files
    delays = []
    for file_path in goodFiles:
        fullFname = file_path
        if not os.path.isabs(file_path):
            fullFname = os.path.join(visit_path, file_path)
        mouse_delay_infos = getMouseDelayInfos(fullFname)
        if mouse_delay_infos is not None:
            delays.append([mouse_delay_infos, fullFname])

    test_delays(delays)

## Compute the kinect-to-mouse delay for the XDF data

In [ ]:
import pyxdf


def getKinectToMouseDelay(data, fileName):
    """Get the delay between the kinect and the mouse data"""

    # find the mouse and kinect markers
    kinect_markers = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "EuroMov-Markers-Kinect"
        and stream["info"]["type"][0] == "Markers"
    ]
    kinect_markers = kinect_markers[0]

    mouse_markers = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "Mouse"
        and stream["info"]["type"][0] == "Markers"
    ]
    mouse_markers = mouse_markers[0]

    # check whether the mouse and kinect markers are from the same hostname
    kinect_hostname = kinect_markers["info"]["hostname"][0]
    mouse_hostname = mouse_markers["info"]["hostname"][0]

    # make a warning if the mouse and kinect markers are not from the same hostname
    message = ""
    if kinect_hostname == mouse_hostname:
        message += (
            "The mouse and kinect markers are from the same computer clock: {}.".format(
                kinect_hostname
            )
        )
    else:
        message += "CAUTION: the mouse and kinect markers are not from the same computer clock! "
        message += "Kinect hostname: {}, ".format(kinect_hostname)
        message += "Mouse hostname: {}.".format(mouse_hostname)

    # get the delay
    kinect_delay = getKinectDelayInfos(fileName)["median_delay"]
    mouse_delay = getMouseDelayInfos(fileName)["median_delay"]
    kinect_to_mouse_delay = mouse_delay - kinect_delay

    # get the distribution of the delays
    kinect_delays_distribution = getKinectDelayInfos(fileName)["delays_distribution"]
    mouse_delays_distribution = getMouseDelayInfos(fileName)["delays_distribution"]

    # return a dict with the delays
    return {
        "kinect_delay": kinect_delay,
        "mouse_delay": mouse_delay,
        "kinect_to_mouse_delay": kinect_to_mouse_delay,
        "kinect_delays_distribution": kinect_delays_distribution,
        "mouse_delays_distribution": mouse_delays_distribution,
        "fileName": fileName,
        "message": message,
    }


if doRunTests:
    fileName = goodFiles[8]
    fileName = goodFiles[17]

    if not os.path.isabs(fileName):
        fileName = os.path.join(visit_path, fileName)

    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    ktm = getKinectToMouseDelay(data, fileName)

    print(
        "The kinect to mouse delay is {} seconds".format(ktm["kinect_to_mouse_delay"])
    )
    print("The kinect delay is {} seconds".format(ktm["kinect_delay"]))
    print("The mouse delay is {} seconds".format(ktm["mouse_delay"]))
    print(
        "The kinect delay distribution is {}".format(ktm["kinect_delays_distribution"])
    )
    print("The mouse delay distribution is {}".format(ktm["mouse_delays_distribution"]))
    print(
        "The kinect delay distribution median is {}".format(
            np.median(ktm["kinect_delays_distribution"])
        )
    )
    print(
        "The mouse delay distribution median is {}".format(
            np.median(ktm["mouse_delays_distribution"])
        )
    )
    print(ktm["message"])

    # make a boxplot of the distribution of the delays
    fig = plt.figure()
    kinect_delay = ktm["kinect_delays_distribution"] * 1000  # in milliseconds
    mouse_delay = ktm["mouse_delays_distribution"] * 1000
    plt.boxplot(
        [kinect_delay, mouse_delay],
        vert=False,
        labels=["Kinect", "Mouse"],
    )
    plt.xlabel("Delay (milliseconds)")
    plt.title("Distribution of the delays\n" + file_parts(fileName)[1])
    plt.show()

#  Compute the Kinect & Mouse sequence of events

We want to integrate all the events (markers) given individually by the Kinect and the Mouse into a single time sequence that combines Kinect + Mouse.

## Define the expected marker sequence (from csv)

The total sequence is the sum of the sequences from the Mouse and Kinect marker marker files (in csv format, in this directory).  
This is the *ground truth* sequence of events, necessary for comparison with the *actual* sequence.

In [ ]:
import glob


def setExpectedMarkerSequence(fileNameXdf):
    fpath, fname, extension = file_parts(fileNameXdf)

    # find all the csv files in the directory that are markers files
    csvFiles = glob.glob("*_m*.csv", root_dir=fpath)

    expectedMarkerSequence = []
    for foundFile in csvFiles:
        fullFname_csv = os.path.join(fpath, foundFile)
        data_csv = readMarkerCsv(fullFname_csv)
        for line in data_csv:
            expectedMarkerSequence.append(line)

    expectedMarkerSequence.sort(key=lambda x: x[0])

    return expectedMarkerSequence, csvFiles


if doRunTests:
    fileName = goodFiles[17]
    fileName = goodFiles[8]

    if not os.path.isabs(fileName):
        fileName = os.path.join(visit_path, fileName)

    expectedMarkerSequence = setExpectedMarkerSequence(fileName)

    print("expectedMarkerSequence:")
    for i in range(len(expectedMarkerSequence)):
        print(
            "  {}: {}".format(
                expectedMarkerSequence[i][0], expectedMarkerSequence[i][-1]
            )
        )

## Define the actual marker sequence (from xdf)


The total sequence is the sum of the sequences from the Mouse and Kinect marker streams.  
We have to make a correction for the "error" in the Kinect markers, which are delayed by a constant amount of time (i.e., the `kinect_to_mouse_delay`).  
This is the *actual* sequence of events, necessary for comparison with the *ground truth* sequence.

In [ ]:
import copy


def getActualMarkerSequence(fileName, data=None):
    if data is None:
        # load the whole xdf file
        data, header = pyxdf.load_xdf(
            filename=fileName,
            synchronize_clocks=True,
            dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
        )
    ktm = getKinectToMouseDelay(data, fileName)

    # we only need the kinect marker stream and the mouse marker stream
    kinectMarkerStream = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "EuroMov-Markers-Kinect"
        and stream["info"]["type"][0] == "Markers"
    ][0]
    mouseMarkerStream = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "Mouse"
        and stream["info"]["type"][0] == "Markers"
    ][0]

    #  do not  modify the original kinect stream ! use deepcopy
    kinectMarkerStream = copy.deepcopy(kinectMarkerStream)

    # We sync the (copy of the) kinect stream with the mouse stream
    kinectMarkerStream["time_stamps"] += ktm["kinect_to_mouse_delay"]
    if "Initial stop" in kinectMarkerStream["time_series"][0][0]:
        # remove the "Initial stop" marker
        kinectMarkerStream["time_stamps"] = kinectMarkerStream["time_stamps"][1:]
        kinectMarkerStream["time_series"] = kinectMarkerStream["time_series"][1:]

    # set the actual marker sequence
    actualMarkerSequence = []
    for stream in [kinectMarkerStream, mouseMarkerStream]:
        for i in range(len(stream["time_stamps"])):
            actualMarkerSequence.append(
                [
                    stream["time_stamps"][i],
                    stream["time_series"][i][0],
                    stream["info"]["name"][0],
                ]
            )

    # sort the markers by timestamp
    actualMarkerSequence.sort(key=lambda x: x[0])

    return actualMarkerSequence


if doRunTests:
    fileName = goodFiles[8]
    # fileName = goodFiles[17]

    if not os.path.isabs(fileName):
        fileName = os.path.join(visit_path, fileName)
    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    actualMarkerSequence = getActualMarkerSequence(fileName, data)

    # print the markers
    print("Actual marker sequence:")
    for marker in actualMarkerSequence:
        print(marker)

# Compare the expected marker sequence with the actual marker sequence

## Create two comparable lists of markers

The csv files contain a list of markers with timestamps that are absolute (i.e., classical datetime timestamps where t = 0 occurs on 1970-01-01 01:00:00).  
The xdf files contain a list of markers with timestamps that are relative (i.e., we do not know when t = 0 occurs).  

To compare the two lists of markers, we have to make them comparable in time: we decide that their first common marker occurs at time = 0.

Below we create two such lists of markers, one from the csv file and one from the xdf file.  



In [ ]:
def getComparableMarkerSequence(fileName, data):
    """
    Get the marker sequence that can be compared with the CSV files
    """

    actualMarkerSequence = getActualMarkerSequence(fileName, data)
    expectedMarkerSequence, csvFiles = setExpectedMarkerSequence(fileName)

    # make the timestamps relative to the first timestamp
    firstTimestamp = actualMarkerSequence[0][0]
    for i in range(len(actualMarkerSequence)):
        actualMarkerSequence[i][0] -= firstTimestamp

    firstTimestamp = expectedMarkerSequence[0][0]
    for i in range(len(expectedMarkerSequence)):
        expectedMarkerSequence[i][0] -= firstTimestamp

    return {
        "actualMarkerSequence": actualMarkerSequence,
        "expectedMarkerSequence": expectedMarkerSequence,
        "csvFiles": csvFiles,
    }


if doRunTests:
    fileName = goodFiles[8]
    # fileName = goodFiles[17]
    if not os.path.isabs(fileName):
        fileName = os.path.join(visit_path, fileName)

    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )

    comparableMarkerSequence = getComparableMarkerSequence(fileName, data)

    # user feedback
    actualMarkerSequence = comparableMarkerSequence["actualMarkerSequence"]
    expectedMarkerSequence = comparableMarkerSequence["expectedMarkerSequence"]
    csvFiles = comparableMarkerSequence["csvFiles"]

    print("expectedMarkerSequence:")
    print("From {} csv files".format(len(csvFiles)))
    for i in range(len(csvFiles)):
        print("  {}".format(csvFiles[i]))

    print("{} markers".format(len(expectedMarkerSequence)))
    for i in range(len(expectedMarkerSequence)):
        print(
            "{:10.3f}, {}".format(
                expectedMarkerSequence[i][0], expectedMarkerSequence[i][1]
            )
        )

    print(":")
    print("actualMarkerSequence:")
    print("From {}".format(os.path.basename(fileName)))
    print("{} markers".format(len(actualMarkerSequence)))
    for i in range(len(actualMarkerSequence)):
        print(
            "{:10.3f}, {}".format(
                actualMarkerSequence[i][0], actualMarkerSequence[i][1]
            )
        )

## Compare the two lists of markers

In [ ]:
def checkMarkerSequence(fileName, data):
    """
    Check whether the marker sequence is correct
    """

    msg_sequence_error = ""
    msg_feedback = ""
    msg_too_big_time_error = ""

    comparableMarkerSequence = getComparableMarkerSequence(fileName, data)

    actualMarkerSequence = comparableMarkerSequence["actualMarkerSequence"]
    expectedMarkerSequence = comparableMarkerSequence["expectedMarkerSequence"]
    csvFiles = comparableMarkerSequence["csvFiles"]

    if doRunTests:
        # NOTE: we save the actual and expected correct sequences into 2 files
        #       so that we can compare them with a diff tool
        fnameActual = "debug/actual.csv"
        fnameExpected = "debug/expected.csv"
        with open(fnameActual, "w") as f:
            for i in range(len(actualMarkerSequence)):
                f.write(
                    "{:10.2f}, {}\n".format(
                        actualMarkerSequence[i][0], actualMarkerSequence[i][1]
                    )
                )
        with open(fnameExpected, "w") as f:
            for i in range(len(expectedMarkerSequence)):
                f.write(
                    "{:10.2f}, {}\n".format(
                        expectedMarkerSequence[i][0], expectedMarkerSequence[i][1]
                    )
                )

        # check line by line whether the expected sequences is the same as the actual sequence
        nValidMarkers = 0
        for i in range(len(expectedMarkerSequence)):
            if actualMarkerSequence[i][1] == expectedMarkerSequence[i][1]:
                nValidMarkers += 1
            else:
                msg_sequence_error += "Error in sequence:\n"
                msg_sequence_error += "  Marker sequence, Line: {}\n".format(i)
                msg_sequence_error += "    Actual  [{}] = {}\n".format(
                    i, actualMarkerSequence[i]
                )
                msg_sequence_error += "    Expected[{}] = {}\n".format(
                    i, expectedMarkerSequence[i]
                )

                break

    # count the number of correct markers
    nExpected = len(expectedMarkerSequence)
    nValidMarkers = 0
    for i in range(len(expectedMarkerSequence)):
        if actualMarkerSequence[i][1] == expectedMarkerSequence[i][1]:
            nValidMarkers += 1

    # check whether the actual and expected correct sequences have the similar diff between two consecutive timestamps
    actualDiff = np.diff([marker[0] for marker in actualMarkerSequence[:i]])
    expectedDiff = np.diff([marker[0] for marker in expectedMarkerSequence[:i]])

    errorinDiff = actualDiff - expectedDiff
    maxError = np.max(np.abs(errorinDiff))

    # check if actualDiff and expectedDiff never differ by more than 5 ms
    time_error_tolerance = 0.005
    if not np.allclose(actualDiff, expectedDiff, atol=time_error_tolerance):
        msg_too_big_time_error += (
            "CAUTION : error in timing is higher than {} ms. ".format(
                time_error_tolerance * 1000
            )
        )

    # user feedback
    # NOTE: We use txt to store the message to be propagated to any caller function
    v = nValidMarkers
    e = nExpected
    msg_feedback += "Correct sequence of {} markers out of {}. ".format(v, e)
    msg_feedback += "The max error is {:3.2f} milliseconds. ".format(maxError * 1000)
    msg_feedback += "{}".format(msg_too_big_time_error)

    if doRunTests:
        print("File: {}".format(fileName))
        if msg_sequence_error != "":
            print("  " + msg_sequence_error)
        if msg_feedback != "":
            print("  " + msg_feedback)
        if msg_too_big_time_error != "":
            print("  " + msg_too_big_time_error)

    return {
        "actualDiff": actualDiff,
        "expectedDiff": expectedDiff,
        "errorinDiff": errorinDiff,
        "msg": msg_feedback + ". " + msg_too_big_time_error,
        "csvFiles": csvFiles,
    }


if doRunTests:
    fileName = goodFiles[8]
    # fileName = goodFiles[17]

    if not os.path.isabs(fileName):
        fileName = os.path.join(visit_path, fileName)

    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )

    sequence_check = checkMarkerSequence(fileName, data)

    # prepare the data for the  user feedback
    fname = os.path.basename(fileName)
    actualDiff = sequence_check["actualDiff"]
    expectedDiff = sequence_check["expectedDiff"]
    errorinDiff = sequence_check["errorinDiff"]
    csvFiles = sequence_check["csvFiles"]

    # make a boxplot of the distribution of the delays
    fig = plt.figure()
    toBoxplot = (actualDiff - expectedDiff) * 1000  # in milliseconds
    csvFilesTxt = "\n".join(csvFiles)
    label = "{}\nvs.\n{}".format(fname, csvFilesTxt)
    plt.boxplot(toBoxplot, vert=False, labels=[label])
    plt.xlabel("Error (milliseconds)")
    plt.title(
        "Difference in duration between two consecutive markers\nxdf vs. csv: {} measures".format(
            len(actualDiff)
        )
    )
    plt.show()

# Get the kinect-to-mouse delay for all xdf files

In [ ]:
import re
from numpy import save


def kinect_to_mouse_delays(goodFiles, visit_path):
    kinect_to_mouse_delay = []
    for file in goodFiles:
        if not os.path.isabs(file):
            file = os.path.join(visit_path, file)
        if file.endswith("xdf"):
            data, header = pyxdf.load_xdf(
                filename=file,
                # load only the streams that we need to get the kinect to mouse delay
                # NOTE: this saves time when loading the xdf file (3.5 to 2.4 seconds for one file)
                select_streams=[
                    {"type": "Markers", "name": "Mouse"},
                    {"type": "Markers", "name": "EuroMov-Markers-Kinect"},
                ],
                synchronize_clocks=True,
                dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
            )
            ktm = getKinectToMouseDelay(data, file)

            # check the marker sequence
            sequence_check = checkMarkerSequence(file, data)

            # restrict the file name to the minimum
            fname = os.path.relpath(ktm["fileName"], visit_path)
            kinect_to_mouse_delay.append(
                [
                    fname,
                    ktm["kinect_to_mouse_delay"],
                    ktm["message"],
                    sequence_check["msg"],
                ]
            )
    return kinect_to_mouse_delay


def save_kinect_to_mouse_delay(ktm_save_fname, kinect_delays):
    # save the kinect_to_mouse_delay to a file
    with open(ktm_save_fname, "w") as f:
        for ktm in kinect_delays:
            toWrite = [str(item) for item in ktm]
            toWrite = "\t".join(toWrite) + "\n"
            f.write(toWrite)
    # get the full path of the file
    fullFname = os.path.abspath(ktm_save_fname)
    print("Delays saved to {}".format(fullFname))
    return fullFname


if doRunTests:
    ktm_delays = kinect_to_mouse_delays(goodFiles, visit_path)

    ktm_fname = os.path.join(visit_path, "kinect_to_mouse_delay.tsv")

    save_kinect_to_mouse_delay(ktm_fname, ktm_delays)

    # load the kinect_to_mouse_delay from a file
    ktm_delays = []
    with open(ktm_fname, "r") as f:
        for line in f:
            tokens = line.strip().split("\t")
            ktm_delays.append(tokens)

    # print the kinect_to_mouse_delay
    print("kinect_to_mouse_delay (from file):")
    for ktm in ktm_delays:
        print("  {}".format(ktm))

# Callable functions

In [ ]:
def kinectToMouseTimeCorrection(visitDataDirectory):
    # load goodFiles into a list
    fullFname_goodFiles = os.path.join(visitDataDirectory, "goodFiles.log")
    goodFiles = []
    with open(fullFname_goodFiles, "r") as f:
        for line in f:
            goodFiles.append(line.strip())

    # set the paths
    visitDirectory = os.path.abspath(visitDataDirectory)
    ktm_fname = os.path.join(visitDirectory, "kinect_to_mouse_delay.tsv")

    # get the kinect to mouse delays for all the goodFiles
    ktm_delays = kinect_to_mouse_delays(goodFiles, visitDirectory)
    save_kinect_to_mouse_delay(ktm_fname, ktm_delays)


if doRunTests:
    visitDirectory = "../dat/ReArm.lnk/ReArm_C1P02"
    visitDirectory = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"
    visitDirectory = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2"

    kinectToMouseTimeCorrection(visitDirectory)